In [1]:
from ultralytics import YOLO
import os
import json

model_path = 'watermarks.pt'
INPUT_DIR = "input_images"
OUTPUT_DIR = "output_images"

model = YOLO(model_path)

def run_inference(image_paths: list):
    try:
        results = model.predict(image_paths, conf=0.1, iou=0)
        saved_paths = [p.replace(INPUT_DIR, OUTPUT_DIR) for p in image_paths]
        
        watermark_status = []
        for idx, result in enumerate(results):
            result.save(saved_paths[idx])
            if len(result.boxes) > 0:
                watermark_status.append(
                    {
                        "image": image_paths[idx],
                        "status": True
                    }
                )
            else:
                watermark_status.append(
                    {
                        "image": image_paths[idx],
                        "status": False
                    }
                )
        with open("result.json", "w", encoding='utf-8') as f:
            json.dump(watermark_status, f, indent=4)

    except Exception as e:
        return e

In [2]:
image_paths = [os.path.join(INPUT_DIR, p) for p in os.listdir(INPUT_DIR)]
inference_result = run_inference(image_paths)


0: 448x640 7 watermarks, 431.5ms
Speed: 20.3ms preprocess, 431.5ms inference, 24.9ms postprocess per image at shape (1, 3, 448, 640)
